In [1]:
import pandas as pd
import glob
import os
import numpy as np
from datetime import datetime
import re

## ETL

In [3]:
# get all the raw csv files in a folder (adjust ur path)
csv_files = glob.glob(r"C:\Users\Luca-PC\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Anaconda3 (64-bit)\Semester 5\APP\Measurements MS200 16g and PCS7 January 2023\X_Data\M1.3 10012023 MS200 controlled by pump\*.csv")


# loop through all files 
for i,filepath in enumerate(csv_files):
    # transformations
        df = pd.read_csv(filepath, sep=';', engine='python')
        df = df.iloc[:, 1:4]
        df = df.truncate(after=511)
        df = df[[' x',' y', ' z']].astype(str).astype(int)
        df["time"] = datetime.strptime(re.sub(r'|c||_|\.|a|s|v', "", os.path.basename(filepath)), '%Y%m%d%H%M%S') # get date from file name
        # safe transformated csv files in a destination file
        output_folder = r"C:\Users\Luca-PC\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Anaconda3 (64-bit)\Semester 5\APP\pump data"
        output_filepath = os.path.join(output_folder, os.path.basename(filepath))
        df.to_csv(output_filepath, sep="," , index=False)

In [4]:
import os
import glob
import pandas as pd

# Pfad zum Verzeichnis, das alle CSV-Dateien enthält
path = r"C:\Users\Luca-PC\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Anaconda3 (64-bit)\Semester 5\APP\pump data"


# Alle CSV-Dateien im Verzeichnis lesen und in eine Liste speichern
all_files = glob.glob(os.path.join(path, "*.csv"))

# Leeres DataFrame erstellen, um alle Daten zu kombinieren
combined_csv = pd.DataFrame()

# CSV-Dateien nacheinander durchgehen und Daten in das DataFrame einfügen
for file in all_files:
    df = pd.read_csv(file)
    combined_csv = pd.concat([combined_csv, df], ignore_index=True)

# Kombinierte CSV-Datei im gleichen Verzeichnis speichern
combined_csv.to_csv(os.path.join(path, "df_pump.csv"), index=False, encoding='utf-8-sig')


## Greate label from label file

In [ ]:
df_valve = pd.read_parquet("df_pump.parquet")
df_value = pd.read_csv("Y_Data.csv", sep=";", low_memory=False, usecols = ['F14002 ValueY','F14002 Time'])
df_value['F14002 Time'] = pd.to_datetime(df_value['F14002 Time'])
df_value['F14002 Time'] = df_value['F14002 Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_value['F14002 Time'] =  pd.to_datetime(df_value['F14002 Time'], format='%Y-%m-%d %H:%M:%S')
df_value['F14002 ValueY'] = df_value['F14002 ValueY'].str.replace(',', '.')
df_value["F14002 ValueY"] = df_value['F14002 ValueY'].astype(float).round(6)
df_value.dropna(inplace=True)
df_value

In [ ]:
df_valve['label']='F14002 ValueY'
for i,s in df_value.iterrows():
    indx= (df_valve['time'] == s["F14002 Time"])
    df_valve.loc[indx,'label']=s['F14002 ValueY']

In [ ]:
df_valve.to_parquet("df_pump.parquet")